## Imports iniciales

Asegúrate de que tienes instalados todos los paquetes y librerías antes de continuar.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import linear_model as lm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

bodyfat = fetch_openml(name='bodyfat', version=1)
df = pd.DataFrame(data= np.c_[bodyfat['data'], bodyfat['target']],
                     columns= bodyfat['feature_names'] + ['target'])
y = df['target']
X = df.loc[:, df.columns != 'target']
print(X.head())
print(X.shape)
print(y.head())
print(y.shape)

# Regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

y_pred = reg.predict(X)
print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicio

Selecciona un par de variables interesantes para poder dibujar el gráfico de la regresión lineal y observa el ajuste gráfico frente a las medidas de rendimiento obtenidas

Para dibujar un gráfico de ajuste con scatter plot puedes usar este código básico:
```python
plt.scatter(X, y,color='g') 
plt.plot(X, y_pred, color='k') 
plt.show()
```

# Regresión polinomial

En Scikit Learn, la regresión polinomial se crea aplicando una transformación a los atributos, generando los nuevos atributos con los coeficientes del polinomio. A continuación, se realiza un ajuste de una regresión lineal.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Por defecto, PolynomialFeatures incluye el término independiente Bias a true
poly_reg = PolynomialFeatures(degree=3) #cuidado con el grado: el número de coeficientes crece exponencialmente con su valor de acuerdo a la matriz de Vandermonde

X_poly = poly_reg.fit_transform(X)

# Ya tenemos las características transformadas, ahora entrenamos el modelo
reg2 = LinearRegression()
reg2.fit(X_poly,y) # Aquí es donde ajustamos los coefientes del modelo y reg2 se actualiza en consecuencia

y_pred = reg2.predict(X_poly)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

Es posible crear un flujo completo de trabajo utilizando la función make_pipeline().

## Regresión Lineal con make_pipeline

Esta función genera la transformación y la aplica al regresor (lineal en nuestro caso) en una única línea.

In [ ]:
from sklearn.pipeline import make_pipeline

reg3 = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
reg3.fit(X,y) 

y_pred = reg3.predict(X) # Fíjate que utilizamos los datos sin transformar para realizar la predicción. El pipeline se encarga de realizar la transformación

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicio
¿Por qué los valores de rendimiento mejoran cuando aumentamos el grado del ajuste polinomial? Pruebe a realizar una validación de training y test (mínimo un Hold-out) y observe de nuevo el efecto de aumentar el grado del polinomio.

**Nota**: Si utiliza particiones de entrenamiento y test, tendrá que aplicar el método fit_transform a los valores X tanto del conjunto de entrenamiento como del conjunto de test.

# Regresión con Splines

En la regresión con splines se aplica la regresión polinómica "a trozos": entre ciertos valores de los atributos de entrada (los nodos o "knots") se genera el ajuste de un regresor polinómico.
Esto supone crear una transformación como la que hemos visto para la regresión polinómica, pero la matriz de transformación es diferente ahora.

Además, hemos de elegir qué estimador polinómico queremos ajustar entre los nodos.

Para condensar el ejemplo, utilizaremos la función make_pipeline() de nuevo.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer

spline = make_pipeline(SplineTransformer(n_knots=8, degree=6), LinearRegression()) #n_knots es el número de puntos de control y degree el grado del polinomio
spline.fit(X, y)

y_pred = spline.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicio

1. Observa el efecto del número de nodos o puntos de control y del grado en el rendimiento del algoritmo.
2. Experimenta con una validación adecuada (Hold-out o validación cruzada) y el uso del pipeline para realizar la validación.
3. Prueba a elegir un atributo de entrada para poder pintar con un plot el resultado del ajuste spline frente a la salida.

# Regresión lineal a trozos (LOESS)

Actualmente, sklearn no incluye este modelo de regresión, aunque el paquete statmodels sí incluye una versión unidimensional.
La incluimos a continuación con un ejemplo.

In [ ]:
#| code-fold: True
from statsmodels.api import nonparametric

X_mono = X['Abdomen'] # La regresión linea a trozos sólo admite una variable independiente en statsmodels
lowess_sm = nonparametric.lowess

y_pred = lowess_sm(y,X_mono.to_numpy(),frac=1./3.,it=3, return_sorted = False)
print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

plt.scatter(X['Abdomen'], y, color = 'red')
plt.plot(X['Abdomen'], y_pred, color = 'blue')
plt.show()

## Ejercicio

Se aprecia como la regresión "va y vuelve". Intente solucionarlo ordenando los datos de entrada (X_mono).

# Generalized Additive Models

De nuevo, sklearn no tiene una implementación nativa de GAMs. Es posible encontrar una buena biblioteca en https://pygam.readthedocs.io/en/latest/

La información para su instalación está en https://pygam.readthedocs.io/en/latest/notebooks/quick_start.html#Install-pyGAM

Recordemos que las GAMs permiten establecer linealidades o no linealidades en diferentes atributos a nuestra elección. PyGAM especifica esto en la creación del modelo.

In [ ]:
#|output: False
!pip install pygam

---

In [ ]:
from pygam import LinearGAM, s, f, l

# Definimos el modelo utilizando las siguientes funciones:
# 1. la función s para indicar que se ajustara una función spline a la variable independiente con el correspondiente índice
# 2. En el caso de usar f, se aplicará un factor de ajuste a la variable independiente con el correspondiente índice
# 3. En el caso de usar l, se aplicará un término lineal a la variable independiente con el correspondiente índice

# Tenemos 14 atributos de entrada en nuestro conjunto, vamos a realizar un ajuste por cada uno de ellos de forma arbitraria
gam = LinearGAM(s(0) + s(1) + f(2) + l(3) + l(4) + l(5) + l(6) + l(7) + l(8) + l(9) + l(10) + l(11) + l(12) + l(13))

gam.fit(X, y)

#Vamos a visualizar el modelo
gam.summary()

---

In [ ]:
y_pred = gam.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios

1. Observe los p-values devueltos por el summary() de nuestra GAM. Aquellos valores más bajos dan pistas de los atributos más importantes. Prueve a variar el ajuste utilizado para esos atributos.
2. Pruebe a ajustar los parámetros de regularización con el método gridsearch() (más info en https://pygam.readthedocs.io/en/latest/notebooks/quick_start.html#Automatically-tune-the-model)

# Gradiente descendente estocástico

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

sgd = SGDRegressor(max_iter=1000, tol=1e-3, penalty='l2', alpha=0.1, eta0=0.01, learning_rate='constant', loss='squared_error', random_state=100)
sgd.fit(X, y)

y_pred = sgd.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

Los resultados obtenidos en el bloque de código anterior son muy pobres, deben estar normalizados.

---

## Normalizando

Vamos a normalizar los datos para que tengan media 0 y desviación típica 1, ya que el uso de **SGD** sin normalizar los datos puede provocar que el modelo no converja

In [ ]:
scaler = StandardScaler()
X_scal = scaler.fit_transform(X)
sgd.fit(X_scal, y)

y_pred = sgd.predict(X_scal)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Pruebe diferentes parámetros max_iter, tol, alpha y eta  de SGDRegressor y observe el comportamiento del regresor.
2. Es muy habitual realizar el escalado y el fit() del SGDRegressor. SKlearn proporciona un pipeline para juntar las dos tareas. Examine la documentación de https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html y aplique el método make_pipeline al ejemplo anterior

# Ridge Regression

Ridge regression es un método de regresión múltiple en escenarios en los que las variables independientes (de entrada) están muy correlacionadas. Resulta especialmente útil para mitigar el problema de la multicolinealidad en la regresión lineal, que suele darse en modelos con un gran número de parámetros.

In [ ]:
from sklearn import linear_model

ridge = linear_model.Ridge(alpha=.5)
ridge.fit(X, y)

y_pred = ridge.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Intente obtener el mismo comportamiento que el regresor lineal con el Ridge regressor
2. De igual forma, es posible emular el comportamiento del Ridge regressor mediante el regresor SGD. Revise las transparencias de teoría e intente obtener dicha equivalencia.

# Lasso regression

::: {.notes}
Lasso se introdujo para mejorar la precisión de las predicciones y la interpretabilidad de los modelos de regresión. Realiza una selección de un subconjunto de las covariables conocidas para su uso en un modelo. 

Previamente, Ridge regression era la técnica más popular para mejorar la precisión de las predicciones. Como hemos visto antes, Ridge Regression mejora el error de predicción reduciendo la suma de los cuadrados de los coeficientes de regresión para que sea inferior a un valor fijo con el fin de reducir el sobreajuste, pero no realiza la selección de covariables y, por lo tanto, no ayuda a que el modelo sea más interpretable.
:::

Lasso es la técnica que aborda los problemas de **Ridge**.

Lasso logra ambos objetivos al forzar la suma del valor absoluto de los coeficientes de regresión a ser inferior a un valor fijo, lo que fuerza a ciertos coeficientes a cero, excluyéndolos del impacto en la predicción. Esta idea es similar a la de Ridge regression, que también reduce el tamaño de los coeficientes; sin embargo, Ridge regression no pone los coeficientes a cero (y, por tanto, no realiza la selección de variables).

In [ ]:
from sklearn import linear_model

lasso = linear_model.Lasso(alpha=.5)
lasso.fit(X, y)

y_pred = lasso.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Tal y como ocurría con el Ridge regressor, puede emular el comportamiento del Ridge regressor mediante el regresor SGD. Revise las transparencias de teoría e intente obtener dicha equivalencia.
2. Observe los coeficientes del ajuste del regresor Lasso con el método **.coef_** e identifique los atributos menos importantes.
3. Con la información del punto 2, podría eliminar los atributos con un peso de cero del conjunto y mejorar el rendimiento (velocidad) de ajuste del modelo sin afectar apenas al rendimiento. Pruébelo.

# Elastic Net

El método Elastic Net supera las limitaciones de LASSO. La regularización de LASSO tiene varias limitaciones. Por ejemplo, en el caso "p grande, n pequeño" (datos de alta dimensión con pocos ejemplos), LASSO selecciona como máximo n variables antes de saturarse. Además, si hay un grupo de variables muy correlacionadas, LASSO tiende a seleccionar una variable del grupo e ignorar las demás. Para superar estas limitaciones, la red elástica añade una parte cuadrática $( ||\beta||^2)$ a la penalización, que corresponde a la penalización de Ridge Regression.

El término de penalización cuadrática añadido hace que la función de pérdida sea fuertemente convexa y, por tanto, tiene un mínimo único. El método de Elastic Net, por tanto, incluye tanto a LASSO como a Ridge Regression: en otras palabras, estos últimos son un caso especial en el que $\lambda_1 = \lambda, \lambda_2 = 0$ o $\lambda_1 = 0, \lambda_2 = \lambda$ dada la regularización de Elastic Net:
$$ \hat{\beta} = \underset{\beta}{\mathrm{argmin}} (||y - X\beta||^2 + \lambda_2||\beta||^2 + \lambda_1||\beta||) $$

---

In [ ]:
from sklearn import linear_model

elastic = linear_model.ElasticNet(alpha=0.5, l1_ratio=0.5)
elastic.fit(X, y)

y_pred = elastic.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Dado que Elastic Net es una "combinación" de Lasso y Ridge, pruebe a modificar el parámetro **l1_ratio** para emular el comportamiento obtenido en los dos modelos mencionados.
2. Observe también como cambian los coeficientes del ajuste de Elastic Net cuando modifica el **l1_ratio**.


## Regresión mediante mínimos cuadrados parciales (PLS regression)

En https://scikit-learn.org/stable/modules/cross_decomposition.html es posible leer las diferentes versiones de este algoritmo. El algoritmo base es PLSCanonical, y el resto de versiones realizan modificaciones al algoritmo base para ganar eficiencia o aplicar regularizaciones.

A diferencia de lo visto en teoría, no vamos a realizar la regresión de PCA+Linear regression y nos centraremos en PLS directamente. Esperaremos a la práctica de preprocesamiento para aprender cómo aplicar PCA en nuestros datos. En cualquier caso, si tienes curiosidad, este tema está tratado en sklearn directamente: https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html

## PLS Canonical

In [ ]:
from sklearn.cross_decomposition import PLSCanonical

#n_components es el número de componentes principales a utilizar
pls = PLSCanonical(n_components=1, tol= 0.001, max_iter=5000) 
pls.fit(X, y)

y_pred = pls.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## PLS regression

Permite hacer regresión de una variable (PLS1 con n_components=1) o varios (PLS2 con n_components > 1).
A diferencia de PLSCanonical, el número de componentes no está limitado por el número de atributos de salida. Se encuentra en el mínimo de $[1, min(\#atributos, \#ejemplos)]$

In [ ]:
from sklearn.cross_decomposition import PLSRegression

#n_components es el número de componentes principales a utilizar
#prueba a aumentar el número de componentes y observa como cambia el error
pls = PLSRegression(n_components=2, tol= 0.001, max_iter=5000) 
pls.fit(X, y)

y_pred = pls.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Con la experiencia de estos dos métodos, prueba a ajustar con PLSSVD los datos: https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSSVD.html#sklearn.cross_decomposition.PLSSVD
2. ¿Cuál es el efecto del número de componentes en PLSRegression en el rendimiento? ¿Hay alguna asíntota de rendimiento? 
3. ¿Qué ocurre con los coeficientes al aumentar el número de componentes?
4. Prueba a aplicar algún tipo de estandarízación en los datos y observa las diferencias en rendimiento.

# Regresión con MLP (Multi-layer Perceptron)

## MLP de SKlearn

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(10,10), activation='relu', solver='sgd', learning_rate='constant', max_iter=500, verbose=True, random_state=100)
mlp.fit(X, y)

y_pred = mlp.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

## Ejercicios
1. Cambia la topología de la red (la cantidad de capas ocultas y el número de neuronas) para intentar mejorar los resultados
2. Puedes ajustar el resto de parámetros (info en https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) para intentar conseguir mejorar los resultados.
3. Juega con el **learning_rate** frente al número de iteraciones y los resultados. Cambiar el **solver** también tiene un impacto importante en la velocidad de convergencia.

## MLP con Keras

Aunque en asignaturas más avanzadas se trabajará con Keras de forma más profunda, aprovechamos aquí para hacer un pequeño inciso.

En Keras, la construcción de la topología de la red para obtener una MLP se hace artesanalmente, añadiendo capas de izquierda a derecha (utilizando un modelo Sequential), comenzando por la primera capa de conexión con los atributos del dataset y terminando con la última neurona de salida.

Para facilitar la tarea vamos a utilizar un wrapper que incorpora Sklearn para trabajar con Keras de la misma forma que hemos hecho con los algoritmos de regresión previos. Queda como tarea al alumno replicar el uso de Keras directamente mostrado en las transparencias de teoría.

No lo ejecuto aquí porque da problemas con Python 3.11.

Recuerda tener instalado:

In [ ]:
#|output: False
!pip install scikeras tensorflow

---

In [ ]:
#|eval: False
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

#vamos a definir la topología del modelo MLP igual al ejemplo con MLPRegressor (2 capas ocultas de 10 neuronas cada una)
def mlp():
    model = Sequential()
    model.add(Dense(10, input_dim=len(X.columns), activation='relu')) #capa de entrada con input_dim = número de atributos de nuestro conjunto de datos
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse']) #para regresión usamos mean_squared_error como función de pérdida
    return model

""""
#Alternativamente, podemos definir la topología del modelo MLP como un vector de capas directamente en lugar de usar el método add
mlp = models.Sequential(
    [
        keras.layers.Dense(10, activation="relu", input_shape=len(X.columns)),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ]
)
mlp.compile(loss="mse", optimizer="adam", metrics=["mse"])

mlp.summary()
"""

---
Ahora podemos utilizar KerasRegressor para crear el modelo y ajustarlo a los datos.

In [ ]:
#|eval: False
mlp_keras = KerasRegressor(model=mlp, epochs=100, batch_size=5, verbose=1)
mlp_keras.get_params()

mlp_keras.fit(X,y) # vamos a entrenar usando CPU. En asignaturas posteriores veremos como entrenar usando GPU

y_pred = mlp_keras.predict(X)

print("Mean squared error: ", mean_squared_error(y, y_pred))
print("Mean absolute error: ", mean_absolute_error(y, y_pred))
print("R2 score: ", r2_score(y, y_pred))

# Consideraciones generales

Dese cuenta que no hemos aplicado ningún tipo de validación en los modelos. Estamos ajustando y evaluando el rendimiento en el conjunto completo, lo que no es nada recomendable. Queda en manos del estudiante aplicar los conocimientos de particionamiento y validación a los diferentes modelos aquí mostrados.

De igual forma, no se ha realizado normalización en la mayoría de casos, lo que es muy recomendable (sobre todo trabajando con SGD). En la sesión de preprocesamiento se tratará con mayor profundidad, pero es necesario dejar constancia de este hecho.

Finalmente, queremos indicar al estudiante que el ajuste de parámetros manual no es lo más óptimo hoy día. Más adelante estudiaremos como ajustar los parámetros de forma automatizada.